In [8]:
import pandas as pd
from pandas.io.json import json_normalize 
import os
import json 

def read_jsonl(path):
    data = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))

    return json_normalize(data)

def nice_name(row):
    if (row["model"] in ["RandomForestClassifier", "ExtraTreesClassifier", "GradientBoostingClassifier"]): 
        model_name = "{} with T = {}".format(row["model"], row["model_params.n_estimators"])
    elif row["model"] == "RiverModel":
        model_name = "{} with {}".format(row["model"], row["river_model"])
    elif (row["model"] == "BiasedProxEnsemble"):
        if int(row["max_trees"]) == 0:
            model_name = "{} with λ = {}, max_depth = {}, modes = {}/{}, stepsize = {}".format(row["model"],row["model_params.l_reg"], row["model_params.max_depth"], row["model_params.init_mode"],row["model_params.next_mode"], row["model_params.step_size"])
        else:
            model_name = "{} with T = {}, max_depth = {}, modes = {}/{}, stepsize = {} with λ = {}".format(row["model"], row["model_params.max_trees"], row["model_params.max_depth"], row["step_size"], row["init_mode"],row["next_mode"], row["l_reg"])
    elif row["model"] == "JaxModel":
        model_name = "{} with T = {}, max_depth = {}, with temp_scaling = {}".format(row["model"], row["model_params.n_trees"], row["model_params.max_depth"], row["model_params.temp_scaling"])
    elif row["model"] == "ProxPruningClassifier":
        model_name = "{} with base {} and λ = {}".format(row["model"], row["model_params.base_estimator"], row.get("model_params.l_reg", "None"))
    elif row["model"] == "AdaBoostClassifier":
        model_name = "{} with base = {} with T = {}".format(row["model"], row["model_params.base_estimator"], row["model_params.n_estimators"])
    elif row["model"] == "PyBiasedProxEnsemble":
        model_name = "{} with reg = {} with l_reg = {}".format(row["model"], row["model_params.regularizer"], row["model_params.l_reg"])
    else:
        model_name = row["model"]
    
    return model_name

#dataset = "covertype"
dataset = "magic"
dataset = os.path.join(dataset, "results")
all_subdirs = [os.path.join(dataset,d) for d in os.listdir(dataset) if os.path.isdir(os.path.join(dataset, d))]
#print(all_subdirs)
latest_folder = max(all_subdirs, key=os.path.getmtime)

#latest_folder = dataset + '/04-01-2021-14:58:21'
df = read_jsonl(os.path.join(latest_folder, "results.jsonl"))
#print(df.head(n=5))
df["nice_name"] = df.apply(nice_name, axis=1)
df = df.round(decimals = 3)
df

KeyError: 'model_params'

In [6]:
from IPython.display import display, HTML
df.columns
tabledf = df[["nice_name", "scores.mean_test_accuracy", "scores.mean_test_loss", "scores.mean_train_accuracy", "scores.mean_train_loss", "scores.mean_n_estimators", "scores.mean_fit_time", "scores.mean_n_parameters"]]
tabledf = tabledf.sort_values(by=['scores.mean_test_accuracy'], ascending = False)
#display(tabledf)
display(HTML(tabledf.to_html()))

,nice_name,scores.mean_test_accuracy,scores.mean_test_loss,scores.mean_train_accuracy,scores.mean_train_loss,scores.mean_n_estimators,scores.mean_fit_time,scores.mean_n_parameters
1,PyBiasedProxEnsemble with reg = prob with l_reg = 0.0,86.293,0.233,87.201,0.229,66.4,77.692,16488.8
0,RandomForestClassifier with T = 64.0,85.977,0.245,86.738,0.242,64.0,0.155,21519.2


In [4]:
from IPython.display import display, HTML
df.columns
tabledf = df[["nice_name", "scores.mean_test_accuracy", "scores.mean_test_loss", "scores.mean_train_accuracy", "scores.mean_train_loss", "scores.mean_n_estimators", "scores.mean_fit_time", "scores.mean_n_parameters"]]
tabledf = tabledf.sort_values(by=['scores.mean_test_accuracy'], ascending = False)
#display(tabledf)
display(HTML(tabledf.to_html()))


,nice_name,scores.mean_test_accuracy,scores.mean_test_loss,scores.mean_train_accuracy,scores.mean_train_loss,scores.mean_n_estimators,scores.mean_fit_time,scores.mean_n_parameters
0,RandomForestClassifier with T = 64,84.479,0.259,84.563,0.258,64.0,0.095,6244.0
